# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：

In [1]:
pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [3]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model"
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit"

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.52G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [4]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [5]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [6]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
print(response[0].split("### Response:")[1])


<think>
嗯，这位患者是一个28岁的程序员，每天工作到很晚，最近出现头晕、脖子疼，偶尔恶心。我需要仔细分析这些症状，找出可能的原因，并给出合适的建议。

首先，头晕通常是由于低血压、贫血、脱水、贫血、颈椎病或前庭功能障碍等引起。这位患者最近工作时间长，可能影响了睡眠，导致血压下降，这可能是一个原因。此外，他可能因为工作压力大而出现贫血，或者脱水的情况也可能导致头晕。

脖子疼可能与颈椎问题有关，也可能与前庭功能有关，因为前庭位于颈部。患者最近可能因为长时间低头工作导致颈部肌肉紧张，或者前庭功能障碍，比如梅毒或前庭神经炎，这也是需要考虑的。

恶心可能与低血糖、脱水或药物副作用有关。这位患者最近是否有饮食不规律，或者是否有服药史？如果他最近饮食有变化，可能会导致低血糖。如果他有其他药物使用，也可能是副作用。

接下来，我应该建议他进行一些初步的检查，比如血压、血糖、心率，以及神经功能测试。如果这些检查发现问题，比如血压下降、血糖异常或神经功能异常，可能需要进一步的治疗。同时，建议他保持良好的生活习惯，比如保证足够的睡眠，避免长时间工作，适当运动，减少咖啡因和酒精的摄入，以改善症状。

此外，如果他出现晕厥、意识模糊、视力模糊等症状，这些可能是低血压导致的，需要紧急处理。如果有持续的头晕、恶心，可能需要药物治疗，比如补充维生素B12或铁剂。

总的来说，这位患者的情况需要综合评估他的生活习惯和可能的健康问题，建议他进行详细的检查以明确病因，并根据检查结果制定相应的治疗计划。同时，提醒他注意劳逸结合，改善生活习惯，以缓解症状。
</think>

这位患者的症状可能由多种因素引起，包括低血压、贫血、脱水、颈椎病或前庭功能障碍等。建议他进行血压、血糖、心率和神经功能测试。根据检查结果，可能需要治疗。同时，建议他保持良好的生活习惯，避免长时间工作，适当运动，减少咖啡因和酒精的摄入。如果出现晕厥、意识模糊或视力模糊等严重症状，应立即就医。对于持续的头晕、恶心，可能需要药物治疗，如补充维生素B12或铁剂。综合评估生活习惯和健康问题后，制定相应的治疗计划，并注意劳逸结合，改善生活习惯。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh


In [8]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [9]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20171 [00:00<?, ? examples/s]

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [10]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [11]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [12]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-3): 4 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [13]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [14]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.077900
2,3.055500
3,2.973100
4,3.049700
5,3.040600
6,2.875300
7,2.809600
8,2.707900
9,2.606500
10,2.477800


TrainOutput(global_step=60, training_loss=2.178019086519877, metrics={'train_runtime': 1750.0704, 'train_samples_per_second': 4.388, 'train_steps_per_second': 0.034, 'total_flos': 3.2609062795739136e+17, 'train_loss': 2.178019086519877})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [26]:
model.save_pretrained("qwen-7b_lora_model")

In [27]:
tokenizer.save_pretrained("qwen-7b_lora_model")

('qwen-7b_lora_model/tokenizer_config.json',
 'qwen-7b_lora_model/special_tokens_map.json',
 'qwen-7b_lora_model/chat_template.jinja',
 'qwen-7b_lora_model/tokenizer.json')

### 9. 测试训练后的生成结果

In [19]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [20]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人已经吃了5天药，感觉腹痛稍微减轻，但还是有些发热，现在右下腹有个包块。我得想想怎么处理。

首先，我得考虑包块是什么。包块可能是个囊肿，也可能是肿瘤。囊肿一般比较大，而且有液体，感觉会更痛。肿瘤呢，可能是个良性的，比如肌纤维瘤，或者恶性，比如转移性肝癌。

然后，包块的位置很重要。右下腹的包块可能是子宫或者卵巢的问题，比如子宫肌瘤。或者，也有可能是肝脏的问题，比如肝癌或者胆囊问题。

我得看看这个包块的大小和质地。如果包块很大，而且质地松软，那可能是个囊肿，比如子宫肌瘤。如果是小一点的包块，质地硬硬的，那可能是肝脏问题，比如肝癌或者胆囊炎。

还有，这个包块的活动性。如果包块可以移动，那可能是子宫肌瘤。如果包块固定，那可能是肝脏问题。

哦，对了，我还要看看腹痛和发热的情况。如果这些症状还在，可能是个恶性肿瘤，比如肝癌。如果症状减轻，可能是个良性肿瘤，比如肌瘤。

哦，对了，别忘了包块的大小和质地，以及腹痛和发热的强度。这些都会影响诊断结果。

总之，我需要根据包块的大小、质地和活动性，以及症状，来判断这个包块的性质，然后决定下一步的处理方式。比如，如果是囊肿，可能需要手术切开；如果是肿瘤，可能需要进一步的影像学检查或手术干预。
</think>
根据病人的情况，右下腹有包块且腹痛稍有减轻但发热，需要考虑包块的性质。如果包块较大且质地松软，可能是子宫肌瘤；如果包块较小且质地硬硬，可能是肝脏问题如肝癌或胆囊炎。此外，包块的活动性也很重要，固定可能提示肝脏问题，而可移动可能提示子宫肌瘤。腹痛和发热的强度也需考虑，如果症状还在，可能提示恶性肿瘤，如肝癌。建议进一步检查包块的大小、质地和活动性，并根据这些信息进行下一步的诊断和治疗。



In [21]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )

    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [22]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这位60岁的男性患者出现了右侧胸疼，而且在X线检查中发现右侧肋膈角消失，这让我想到他可能有肺结核，而且可能还有右侧胸腔积液。嗯，现在他需要进行检查，看看胸水的性质。哦，对了，要了解胸水的性质，首先得考虑一下他平时的饮食情况。如果他吃了很多高脂肪的食物，比如油炸、肥肉，或者吃得太咸，那么这些食物可能会影响到他的胸水。这样，他需要做个脂肪酸分析，看看这些脂肪酸是什么样的。

哦，对了，脂肪酸分析可以告诉我胸水中的脂肪酸是什么样的，这样就能了解他是否有摄入过多脂肪或者盐分。另外，脂肪酸分析还能告诉我这些脂肪酸的来源，是不是来自他平时的饮食，比如肥肉、油炸食品，或者是其他地方。

不过，除了饮食，我还得想想其他可能影响胸水脂肪酸的因素。比如，他有没有长期服用了某些药物，或者有其他疾病，比如肾病、糖尿病，这些都可能影响脂肪酸的分解。嗯，所以脂肪酸分析能够帮助我们排除这些药物和疾病的影响，让我们更清楚地了解他胸水的脂肪酸组成。

哦，对了，还有一个因素就是呼吸。如果他的呼吸功能不太好，比如呼吸不畅或者有呼吸困难，那么他的胸水脂肪酸分解可能也会受到影响。嗯，脂肪酸分析可以帮助我们了解这些呼吸功能是否正常。

哦，对了，还有一点，如果他的血液中有过多的代谢产物，比如尿素、肌酸、肌酐，这些代谢产物可能会影响脂肪酸的分解和合成。所以，我们还需要结合尿素、肌酸、肌酐的水平来综合判断。

哦，对了，还有他体内的糖代谢情况，比如葡萄糖、糖原和肝糖原的水平。这些糖代谢产物可能会影响到脂肪酸的分解，所以这也需要考虑进去。

嗯，总之，脂肪酸分析可以帮我从饮食、药物、呼吸、代谢等多个方面，了解他的胸水脂肪酸的组成和分解情况。这样，我就能更全面地了解他的胸水性质了。嗯，这样看来，脂肪酸分析确实是我们了解胸水性质的一个非常重要的检查方法。
</think>
对于一名66岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，实验室检查中对了解胸水的性质最有帮助的检查是脂肪酸分析。脂肪酸分析可以揭示胸水中的脂肪酸组成，帮助了解其来源，如饮食中的脂肪和盐分摄入。此外，脂肪酸分析还能排除药物、呼吸功能、代谢产物和糖代谢等因素的影响，从而更全面地了解胸水性质。因此，脂肪酸

In [23]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位28岁的程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心，这让我想到他可能有某种疾病。首先，他常年熬夜，长期睡眠不足，这可能会导致一些问题，比如头晕或者恶心。嗯，这让我想到他可能有神经性头晕，或者是一些睡眠相关的疾病。

嗯，神经性头晕，听起来有点像他现在的情况。他最近感觉头晕目眩，甚至有点恶心，这可能跟他的神经系统有关。不过，他现在没有明显的疼痛，也没有其他症状，比如恶心。嗯，这让我有点犹豫，因为有时候即使没有疼痛，也有可能是神经系统的异常反应。

嗯，让我再想想，他最近有没过什么特殊经历？比如突然感染或者受到过度的刺激？嗯，没有，他只是感觉头晕，而且最近没有特别的不适。嗯，可能是神经性头晕，但不确定。

嗯，另外，还有其他可能性吗？比如，他可能有某种神经系统的疾病，比如脑供血不足或者脑水肿。嗯，嗯，脑供血不足会导致头晕，而且可能有恶心，但通常也会伴随有头痛，而他现在没有头痛。嗯，这让我觉得可能不是脑供血不足。

嗯，再想想，他有没有其他症状？嗯，没有。嗯，他只是感觉头晕目眩，而且最近没有明显疼痛，也没有其他症状。嗯，这让我更倾向于神经性头晕。

嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯，嗯

In [24]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
28岁的男性程序员，常年熬夜，最近感觉头晕目眩，甚至有点恶心。嗯，这让我想到，可能是他最近的作息不规律导致的。长时间熬夜，身体的生物钟可能被打乱，导致各种不适。

首先，头晕目眩，这让我想到血压异常。长时间的睡眠不足可能导致血压升高，尤其是如果他最近压力大，可能真的血压偏高。高血压是一种常见的原因，但也有其他可能性。

然后是恶心，这让我想到消化系统问题，比如胃炎或者胃食管反流。长期熬夜可能影响胃酸分泌，导致胃酸不足，进而引发恶心。不过，这和头晕目眩不太直接相关。

还有一种可能性是神经性眩晕，这通常和中耳或前庭系统有关，尤其是长时间的睡眠问题。这种情况下，即使没有血压升高，也可能导致头晕目眩。

再想想，他最近压力大，可能导致身体的血压升高，从而引发头晕目眩。不过，这和消化系统问题不太一样。

如果他最近有高血压症状，比如头晕、头痛、口渴、多尿，那很可能就是高血压。但根据他的症状，这可能不太明显。

嗯，嗯，综合来看，可能还是血压异常更有可能，尤其是结合他的症状。不过，这也需要更多检查来确认。

总之，结合他的症状和生活习惯，高血压的可能性还是挺大的。
</think>
根据患者描述的症状和生活习惯，高血压的可能性较大。长期熬夜可能导致血压升高，尤其是当压力增加时。为了确认这一点，建议进行血压监测和进一步的体格检查，以确认是否存在高血压或其他相关症状。同时，建议患者注意改善生活习惯，如保证充足的睡眠，以缓解症状。


In [32]:
from google.colab import drive
drive.mount('/content/drive')

# !cp -r qwen_7b_lora_model /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat 'qwen_7b_lora_model': No such file or directory


In [36]:
!ls -lah /content

total 48K
drwxr-xr-x 1 root root 4.0K Sep 16 21:18 .
drwxr-xr-x 1 root root 4.0K Sep 16 20:14 ..
drwxr-xr-x 4 root root 4.0K Sep 15 17:50 .config
drwx------ 6 root root 4.0K Sep 16 21:18 drive
drwxr-xr-x 4 root root 4.0K Sep 16 20:34 huggingface_tokenizers_cache
drwxr-xr-x 3 root root 4.0K Sep 16 21:05 outputs
drwxr-xr-x 2 root root 4.0K Sep 16 21:05 qwen-1.5b_lora_model
drwxr-xr-x 2 root root 4.0K Sep 16 21:05 qwen-7.5b_lora_model
drwxr-xr-x 2 root root 4.0K Sep 16 21:16 qwen-7b_lora_model
-rw-r--r-- 1 root root  161 Sep 16 20:32 requirements-colab.txt
drwxr-xr-x 1 root root 4.0K Sep 15 17:50 sample_data
drwxr-xr-x 3 root root 4.0K Sep 16 20:33 unsloth_compiled_cache


In [37]:
!cd /content && zip -r qwen-7b_lora_model.zip qwen-7b_lora_model

  adding: qwen-7b_lora_model/ (stored 0%)
  adding: qwen-7b_lora_model/adapter_config.json (deflated 57%)
  adding: qwen-7b_lora_model/special_tokens_map.json (deflated 70%)
  adding: qwen-7b_lora_model/chat_template.jinja (deflated 75%)
  adding: qwen-7b_lora_model/tokenizer_config.json (deflated 88%)
  adding: qwen-7b_lora_model/tokenizer.json (deflated 81%)
  adding: qwen-7b_lora_model/README.md (deflated 65%)
  adding: qwen-7b_lora_model/adapter_model.safetensors (deflated 8%)
